# Stabilität Eulerverfahren

Aus der Vorlesung ist bekannt, dass sich bestimmte DGL mit impliziten Verfahren besser lösen lassen als mit expliziten Verfahren, und dass wir diese DGL als **steif** bezeichnen.

Außerdem haben wir gelernt, dass das vor allem an der hohen Lipschitzkonstante liegt, die für explizite Verfahren eine sehr kleine Schrittweite bedeutet $\left(h<\frac{2}{\lambda}\right)$.

## Stabilität für die Dahlquist-Gleichung

Um ein Gefühl dafür zu bekommen, wann eine DGL steif ist, und wie das die Performanz der expliziten Löser beeinflusst, schauen wir uns die Dahlquist-Gleichung, also die DGL der Form $y'=-λy,λ>0,y_0=1$ mit der exakten Lösung $y(t)=e^{(-λt)}$, an und variieren den Faktor λ sowie die Schrittweite h.

In [ ]:
%% file dahlquist.m
%% Implizites vs. Explizites Eulerverfahren

clearvars

y0 = 1;
tspan = [0,2];
lambda_selection = [0.1, 1, 4, 10, 16];

% Parameter fuer die graphische Ausgabe
p = 1;
m = length(lambda_selection);

% Schrittweite fuer implizite Loesung
h_i = 0.1;


for lambda = lambda_selection

    %% Variation der Schrittweite fuer explizite Loesung
    h_selection = [0.5 * 2/abs(lambda), 1.01 * 2/abs(lambda)];
    n = length(h_selection);
    
    %% Explizite Form der DGL
    f = @(t,y) -lambda * y;

    for h = h_selection
        
        %% Loesung mit explizitem Eulerverfahren
        if h > 0.5
            h = 0.5;
        end
        t = tspan(1):h:tspan(2);
        y_e = zeros(size(t));
        y_e(1) = y0;
        for i = 2:length(t)
            y_e(i) = y_e(i-1) + h * f(t(i-1),y_e(i-1));
        end

        %% Loesung mit implizitem Eulerverfahren
        t_i = tspan(1):h_i:tspan(2);
        y_i=zeros(size(t_i));
        y_i(1)=y0;
        for i = 2:length(t_i)
            y_i(i) = y_i(i-1) / (1+h_i*lambda);
        end

        %% Graphische Ausgabe
        subplot(m,n,p);
        plot(t_i,y_i,'x-',t,y_e,'x-',t,exp(-lambda*t));
        if lambda > 0
            ylim([0 1])
        end
        xlim([0 2])
        title(sprintf('lambda = %.1f, h = %.3f', lambda, h))
        p = p + 1;
        
    end
end

### Aufgabe

Variieren Sie nun die Parameterauswahl in `lambda_selection` und `h_selection`.

- Wie ändert sich die Lösung?
- Was passiert, wenn `h` unabhängig von `lambda` gewählt wird? Was, wenn `h` für das explizite Verfahren ganz knapp kleiner als $\frac{2}{|\lambda|}$ ist?
- Wann ist das explizite Verfahren nicht mehr hilfreich? Wie 'steif' sieht die zugehörige Funktion aus?

## Stabilität für andere Gleichungen

Ein weiteres klassisches Stabilitätsproblem ist die inhomogene Prothero-Robinson-Gleichung mit $y'=-λ(y-g)+g',λ>0,y_0=1$. $g(t)$ ist eine glatte Funktion. Die exakte Lösung ist $y(t)=g(t)+e^{(-λt)} (y_0-g_0)$. Da $e^{(-λt)}$ für $λ≫1$ schnell gegen 0 geht, sollte die Lösung schnell gegen g(t) konvergieren.

Für $g(t) = const.$ lautet die DGL $y' = -\lambda (y - g)$ mit der Lösung $y(t) = g + e^{(-λt)}$, die gegen $g$ konvergiert. Diese lässt sich leicht für das implizite Eulerverfahren umformen zu $y_{k+1} = \frac{y_k + h\lambda g}{1+h\lambda}$.

In [ ]:
%% file prothero.m
%% Implizites vs. Explizites Eulerverfahren

clearvars

y0 = 1;
tspan = [0,4];
lambda_selection = [0.1, 2, 4, 10, 16];
g = @(t) 4;
g0 = g(0);

% Parameter fuer die graphische Ausgabe
p = 1;
m = length(lambda_selection);

% Schrittweite fuer implizite Loesung
h_i = 0.1;


for lambda = lambda_selection

    %% Variation der Schrittweite fuer explizite Loesung
    h_lim = 2/abs(lambda);
    h_selection = [0.5 * h_lim, 0.9 * h_lim, 1.1 * h_lim];
    n = length(h_selection);
    
    %% Explizite Form der DGL mit g nach finiter Differenz abgeleitet
    f = @(t,y) - lambda* (y - g(t)) + (g(t+h_i)-g(t-h_i))/(2*h_i);

    for h = h_selection
        
        %% Loesung mit explizitem Eulerverfahren
        t = tspan(1):h:tspan(2);
        y_e = zeros(size(t));
        y_e(1) = y0;
        for i = 2:length(t)
            y_e(i) = y_e(i-1) + h * f(t(i-1),y_e(i-1));
        end

        %% Loesung mit implizitem Eulerverfahren
        t_i = tspan(1):h_i:tspan(2);
        y_i=zeros(size(t_i));
        y_i(1)=y0;
        for i = 2:length(t_i)
            y_i(i) = ( y_i(i-1) + h_i*lambda*g(t_i(i))) / (1+h_i*lambda);
        end

        %% Graphische Ausgabe
        subplot(m,n,p);
        plot(t_i,y_i,'x-',t,y_e,'x-',t_i, g(t_i) + exp(-lambda*t_i)*(y0-g0));
        %xlim([0 tspan(2)])
        %ylim([-4 4])
        title(sprintf('lambda = %.1f, h = %.3f', lambda, h))
        p = p + 1;
        
    end
end

### Aufgabe

Variieren Sie nun `g`. Können Sie $g(t) = 4 t^2$ implementieren?